# Convolutional Neural Network (CNN)

In [1]:
# Python Version
from platform import python_version
print(python_version())

3.7.6


## Dataset to Computer Vision

Using CIFAR10 from https://www.cs.toronto.edu/~kriz/cifar.html 

## CNN: Feature Learning e Classification

In [3]:
!pip install -q -U watermark
!pip install -q torchsummary

In [2]:
# Import Packages
import torch 
import torch.nn as nn 
import numpy as np 
import torchvision 
import torch.optim as optim 
import torch.nn.functional as F # package with conv2d
from torchsummary import summary 
from torchvision import datasets 
from torchvision import transforms 
from torch.utils.data.dataloader import DataLoader 
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data'Snow" --iversions

torch       1.5.1
autopep8    1.4.4
numpy       1.18.1
json        2.0.9
torchvision 0.6.1
Data'Snow


## Architeture Model

In [4]:
# Convolutional Layer Example
nn.Conv2d(3, 16, 3, stride=2, padding=(2,1))

Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(2, 1))

In [5]:
# Class to Instance the Model
class ModeloCNN(nn.Module):
    
    # Class Constructor
    def __init__(self):
        super().__init__()
        
        # Input Layer
        self.conv1 = nn.Conv2d(3,16,3,padding=1)
        
        # First Hidden Layer
        self.conv2 = nn.Conv2d(16,32,3,padding=1)
        
        # Second Hidden Layer
        self.conv3 = nn.Conv2d(32,64,3,padding=1)
        
        # Polling Layer
        self.pool = nn.MaxPool2d(2,2)
        
        # Linear Transformation 1 (Full Connected Layer)
        self.linear1 = nn.Linear(64*4*4, 512)
        
        # Linear Transformation 2 (Full Connected Layer)
        self.linear2 = nn.Linear(512, 10)
        
        # Dropout layer to Stadartization
        self.dropout = nn.Dropout(p=0.3)

    # Method to foward network
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1,64 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [6]:
# Preparing Transformation to Data
transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [7]:
# Spliting data between trainning and testing
train_data = datasets.CIFAR10('CIFAR10', 
                              train=True, 
                              download=True, 
                              transform=transformations)

Files already downloaded and verified


In [8]:
# Carrega os Dados de Teste
test_data = datasets.CIFAR10('CIFAR10', 
                             train = False, 
                             download = True, 
                             transform = transformations)

Files already downloaded and verified


In [9]:
# Tamanho dos Datasets
len(train_data), len(test_data)

(50000, 10000)

In [10]:
# Validation Data: size 0.2
validation_size = 0.2

In [13]:
# Trainning data size 
training_size = len(train_data)
print(training_size)

50000


In [14]:
# Index
indices = list(range(training_size))
np.random.shuffle(indices)
index_split = int(np.floor(training_size * validation_size))
print(index_split)

10000


In [15]:
# Index to training and validation data
validation_indices, training_indices = indices[:index_split] , indices[index_split:]

In [17]:
# Training and validation sample
trainning_sample = SubsetRandomSampler(training_indices)
validation_sample = SubsetRandomSampler(validation_indices)
print(validation_sample)

In [18]:
# hyperparams
batch_size = 16 
n_epochs = 30 
learning_rate = 0.01 

In [20]:
# Generate final sample data
train_loader = DataLoader( train_data, batch_size = batch_size,sampler=trainning_sample)
valid_loader = DataLoader( train_data, batch_size = batch_size,sampler=validation_sample)
test_loader = DataLoader( test_data, batch_size = batch_size)

In [21]:
# Model Optimiation
# Creating model from Instace Class
model = ModeloCNN()

In [22]:
print(model)

ModeloCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [24]:
# Identifier Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device.type

'cpu'

In [25]:
# Send model to GPU or CPU
model = model.to(device)

In [26]:
# Defining cost function with crossentropy
criterion = nn.CrossEntropyLoss()

In [27]:
# Creating Optimzer Backward (Backpropagation)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [28]:
# Accuracy Function
def accuracy(preds, y):
    _, pred = torch.argmax(preds,1)
    correct = pred.eq(target.data.view_as(pred))
    acc = correct.sum()/len(correct)
    return acc 

In [30]:
# Trainning Model (Loop)

for epoch in range(1, n_epochs+1):
    # Initialize error in each epoch
    train_loss = 0
    valid_loss = 0
    
    # Create object of trainninf from model
    model.train()
    
    # For each data batch execute trainning
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    # Create object to evaluate model
    model.eval()
    
    # For each data batch calculate evaluation
    for batch_idx, (data, target) in enumerate(valid_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        valid_loss = loss.item()*data.size(0)
        
    # Calculate error in epoch
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    
    # Print
    print(f'| Epoch: {epoch:02} | Erro em Treino: {train_loss:.3f} | Erro em Validação: {valid_loss:.3f} |')

KeyboardInterrupt: 